<a href="https://colab.research.google.com/github/alik604/MNIST_Brain/blob/master/Epileptic_Seizure_Recognition_4_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Epileptic Seizure Recognition - 4 classes 

### By Khizr Ali Pardhan [alik604](https://github.com/alik604/ReadMe)

converting to Tensorflow.keras cause my NN to 30% acc -> 20% acc.... 


In [52]:
import pandas as pd 
import numpy as np 

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import * 

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [53]:
url = 'https://raw.githubusercontent.com/alik604/MNIST_Brain/master/Data/Epileptic%20Seizure%20Recognition.csv'
hddr = ['username','X1','X2','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','X22','X23','X24','X25','X26','X27','X28','X29','X30','X31','X32','X33','X34','X35','X36','X37','X38','X39','X40','X41','X42','X43','X44','X45','X46','X47','X48','X49','X50','X51','X52','X53','X54','X55','X56','X57','X58','X59','X60','X61','X62','X63','X64','X65','X66','X67','X68','X69','X70','X71','X72','X73','X74','X75','X76','X77','X78','X79','X80','X81','X82','X83','X84','X85','X86','X87','X88','X89','X90','X91','X92','X93','X94','X95','X96','X97','X98','X99','X100','X101','X102','X103','X104','X105','X106','X107','X108','X109','X110','X111','X112','X113','X114','X115','X116','X117','X118','X119','X120','X121','X122','X123','X124','X125','X126','X127','X128','X129','X130','X131','X132','X133','X134','X135','X136','X137','X138','X139','X140','X141','X142','X143','X144','X145','X146','X147','X148','X149','X150','X151','X152','X153','X154','X155','X156','X157','X158','X159','X160','X161','X162','X163','X164','X165','X166','X167','X168','X169','X170','X171','X172','X173','X174','X175','X176','X177','X178','y']

raw = pd.read_csv(url, low_memory=False)
raw.columns = hddr

In [54]:
raw.head()

,username,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


[Data source - kaggle](https://www.kaggle.com/harunshimanto/epileptic-seizure-recognition)
* [Data creator - university](https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition)  
* [Full Data](http://epileptologie-bonn.de/cms/front_content.php?idcat=193&lang=3&changelang=3) 

```
y contains the category of the 178-dimensional input vector. Specifically y in {1, 2, 3, 4, 5}:
  5 - eyes open, means when they were recording the EEG signal of the brain the patient had their eyes open
  4 - eyes closed, means when they were recording the EEG signal the patient had their eyes closed
  3 - Yes they identify where the region of the tumor was in the brain and recording the EEG activity from the healthy brain area
  2 - They recorder the EEG from the area where the tumor was located
  1 - Recording of seizure activity
```
I aussume first column is 'username' not 'Unnamed', sample cell `X8.V1.322	`

In [55]:
s = raw['username'].str.split('.')
people = [i[0] for i in s.values]
raw['username'] = [i.argmax() for i in CountVectorizer().fit_transform(people).toarray()]


raw.head(1)

,username,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,13,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4


In [86]:
X = raw.drop(['y' , 'username'], axis = 1) # would be nice to keep 'username', but i need a sequence 
y = raw.y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [87]:
y_train , y_test = y_train.to_numpy(), y_test.to_numpy()

mms = MinMaxScaler()
X_train_trans = mms.fit_transform(X_train)
X_test_trans = mms.transform(X_test)

maxlen = 50#240
seqTrain=sequence.pad_sequences(sequences= X_train_trans, maxlen=maxlen, padding='post', dtype='float32', truncating='post')
seqTest=sequence.pad_sequences(sequences= X_test_trans, maxlen=maxlen, padding='post', dtype='float32', truncating='post')

seqTrain = seqTrain.reshape(seqTrain.shape[0],seqTrain.shape[1],1)
seqTest = seqTest.reshape(seqTest.shape[0],seqTest.shape[1],1)

enc = OneHotEncoder(handle_unknown='ignore')
y_train_oneHot, y_test_oneHot = enc.fit_transform(y_train.reshape(-1, 1)), enc.transform(y_test.reshape(-1, 1))

# y_train_c, y_test_c= to_categorical(y_train), to_categorical(y_test)

## ML - Basic & vanilla_ANN

In [66]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier

DTC = DecisionTreeClassifier() 
RFC = RandomForestClassifier(n_estimators=25, random_state=1)
ETC = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False)

eclf = VotingClassifier(estimators=[('DecisionTreeClassifier', DTC), ('RandomForestClassifier', RFC),('ExtraTreesClassifier',ETC)], voting='hard')
for clf, label in zip([DTC, RFC,ETC, eclf], ['DecisionTreeClassifier', 'RandomForestClassifier', 'ExtraTreesClassifier', 'Ensemble']): 
    _ = eclf.fit(X_train_trans,y_train)
    pred = eclf.score(X_test_trans,y_test)
    print("Acc: %0.10f [%s]" % (pred,label))

Acc: 0.6111304348 [DecisionTreeClassifier]
Acc: 0.6166956522 [RandomForestClassifier]
Acc: 0.6041739130 [ExtraTreesClassifier]
Acc: 0.6146086957 [Ensemble]


```
Acc: 0.6184347826 [DecisionTreeClassifier]
Acc: 0.6250434783 [RandomForestClassifier]
Acc: 0.6139130435 [ExtraTreesClassifier]
Acc: 0.6146086957 [Ensemble]
```
better that if we did not use `MinMaxScaler`


In [59]:
!pip3 install xgboost
from xgboost import XGBClassifier
gb = XGBClassifier(tree_method='gpu_hist',n_jobs =8, max_depth=8,learning_rate=0.01,verbosity=0,n_estimators=300)
f = gb.fit(X_train_trans,y_train)

pred = gb.predict(X_test_trans)

def benchmark(pred, y_test = y_test):
  print( "hamming_loss: \t\t"    ,round(hamming_loss(y_test,pred),3))
  print( "precision_score: \t"   ,round(precision_score(y_test,pred,average='micro'),3))
  print( "recall_score: \t\t"    ,round(recall_score(y_test,pred ,average='micro'),3))
  print("------------------------------")
  print( "accuracy_score: \t"    ,round(accuracy_score(y_test,pred),3))

benchmark(pred)
# 0.607

In [88]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)#y_train.reshape(-1, 1))
y_test = to_categorical(y_test)

dim = X_train_trans.shape[1]
dim

178

In [72]:
classifier = Sequential()

classifier.add(Dense(175, activation='relu', input_dim=dim))
classifier.add(Dense(80, activation='relu'))
classifier.add(Dense(40, activation='relu'))
classifier.add(Dense(6, activation='sigmoid'))

classifier.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =['accuracy'])

history = classifier.fit(X_train_trans,y_train, batch_size=8, epochs=10, validation_data=(X_test_trans,y_test)).history
# epoch = 30
# 2s 252us/step - loss: 1.4459 - acc: 0.3089 - val_loss: 1.4135 - val_acc: 0.3273


Train on 8625 samples, validate on 2875 samples
Epoch 1/10
8625/8625 [==============================] - 2s 200us/sample - loss: 1.6120 - accuracy: 0.1988 - val_loss: 1.6094 - val_accuracy: 0.1993
Epoch 2/10
8625/8625 [==============================] - 2s 175us/sample - loss: 1.6095 - accuracy: 0.2020 - val_loss: 1.6094 - val_accuracy: 0.1993
Epoch 3/10
8625/8625 [==============================] - 1s 173us/sample - loss: 1.6095 - accuracy: 0.2008 - val_loss: 1.6094 - val_accuracy: 0.1927
Epoch 4/10
8625/8625 [==============================] - 2s 175us/sample - loss: 1.6094 - accuracy: 0.1983 - val_loss: 1.6094 - val_accuracy: 0.1927
Epoch 5/10
8625/8625 [==============================] - 2s 180us/sample - loss: 1.6094 - accuracy: 0.2024 - val_loss: 1.6094 - val_accuracy: 0.1927
Epoch 6/10
8625/8625 [==============================] - 2s 178us/sample - loss: 1.6094 - accuracy: 0.2028 - val_loss: 1.6094 - val_accuracy: 0.1927
Epoch 7/10
8625/8625 [==============================] - 2s 200us

## ML - Fancy architextures

In [70]:
classifier = Sequential()

classifier.add(Dense(175, activation='relu', input_dim=dim))
classifier.add(Dense(250, activation='relu'))
classifier.add(Dense(250, activation='relu'))
classifier.add(Dense(50, activation='relu'))
classifier.add(Dense(6, activation='sigmoid'))

classifier.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =['accuracy'])

history = classifier.fit(X_train_trans,y_train, batch_size=8, epochs=30, validation_data=(X_test_trans,y_test)).history
# epoch = 30
# 2s 252us/step - loss: 1.4459 - acc: 0.3089 - val_loss: 1.4135 - val_acc: 0.3273


Train on 8625 samples, validate on 2875 samples
Epoch 1/30
8625/8625 [==============================] - 2s 247us/sample - loss: 4.3934 - accuracy: 0.2016 - val_loss: 4.3778 - val_accuracy: 0.2052
Epoch 2/30
8625/8625 [==============================] - 2s 225us/sample - loss: 4.3208 - accuracy: 0.2003 - val_loss: 4.3729 - val_accuracy: 0.2000
Epoch 3/30
8625/8625 [==============================] - 2s 232us/sample - loss: 4.3195 - accuracy: 0.1991 - val_loss: 4.3738 - val_accuracy: 0.2000
Epoch 4/30
8625/8625 [==============================] - 2s 232us/sample - loss: 4.3200 - accuracy: 0.2026 - val_loss: 4.3816 - val_accuracy: 0.1927
Epoch 5/30
8625/8625 [==============================] - 2s 275us/sample - loss: 4.3200 - accuracy: 0.1976 - val_loss: 4.3734 - val_accuracy: 0.2000
Epoch 6/30
8625/8625 [==============================] - 2s 267us/sample - loss: 4.3202 - accuracy: 0.1991 - val_loss: 4.3742 - val_accuracy: 0.2000
Epoch 7/30
8625/8625 [==============================] - 2s 268us

In [92]:
classifier = Sequential()

classifier.add(LSTM(175,  activation='relu'))
classifier.add(Dense(250, activation='relu'))
classifier.add(Dense(250, activation='relu'))
classifier.add(Dense(50,  activation='relu'))
classifier.add(Dense(6,   activation='sigmoid'))

classifier.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =['accuracy'])

history = classifier.fit(seqTrain,y_train, batch_size=32, epochs=5, validation_data=(seqTest,y_test)).history
# epoch = 30
# 2s 252us/step - loss: 1.4459 - acc: 0.3089 - val_loss: 1.4135 - val_acc: 0.3273


Train on 8625 samples, validate on 2875 samples
Epoch 1/5
8625/8625 [==============================] - 11s 1ms/sample - loss: 1.6192 - accuracy: 0.1998 - val_loss: 1.6094 - val_accuracy: 0.2052
Epoch 2/5
8625/8625 [==============================] - 15s 2ms/sample - loss: 1.6094 - accuracy: 0.1983 - val_loss: 1.6094 - val_accuracy: 0.2052
Epoch 3/5
8625/8625 [==============================] - 14s 2ms/sample - loss: 1.6094 - accuracy: 0.1983 - val_loss: 1.6094 - val_accuracy: 0.2052
Epoch 4/5
8625/8625 [==============================] - 14s 2ms/sample - loss: 1.6094 - accuracy: 0.1983 - val_loss: 1.6094 - val_accuracy: 0.2052
Epoch 5/5
8625/8625 [==============================] - 14s 2ms/sample - loss: 1.6094 - accuracy: 0.1983 - val_loss: 1.6094 - val_accuracy: 0.2052


In [93]:
seqTrain.shape

(8625, 50, 1)